In [2]:
import pandas as pd
import arcpy
from arcgis import GIS
gis = GIS()
import json

In [3]:
fnSegments = 'data/segments/Segments_WF - 2023-08-01.shp'

seg_cols = ['SEGID','SHAPE']

# filter by PLANAREA in segments shapefile
filterPlanArea = ['WFRC','MAG'] # must be an array... if only single item, the still include []

In [4]:
# read in segments, filter, and select only key columns
sdfSegments = pd.DataFrame.spatial.from_featureclass(fnSegments)
sdfSegments = sdfSegments[sdfSegments['PLANAREA'].isin(filterPlanArea)]
sdfSegments = sdfSegments[seg_cols]
display(sdfSegments)

,SEGID,SHAPE
23,0006_146.9,"{""paths"": [[[413442.55030000024, 4422753.7282]..."
24,0006_149.9,"{""paths"": [[[418330.7999999998, 4422866], [418..."
25,0006_150.6,"{""paths"": [[[419421.8803000003, 4422872.2963],..."
26,0006_152.6,"{""paths"": [[[422596.89969999995, 4422889.2949]..."
27,0006_152.9,"{""paths"": [[[423103.9000000004, 4422915.9], [4..."
...,...,...
8721,UTA_7121,"{""paths"": [[[424055.6705156177, 4511848.984225..."
8722,UTA_7132,"{""paths"": [[[424262.6891235444, 4508606.236747..."
8723,UTA_7316,"{""paths"": [[[420773.38413784996, 4495314.95905..."
8724,UTA_7320,"{""paths"": [[[419033.24434054515, 4494684.05276..."


In [5]:
# read in intermediate data
dfModVolAdj = pd.read_csv('intermediate/model-volumes-adjusted.csv')
dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'M' + dfModVolAdj['YEAR']

dfModVolAdj_pvModAadt = dfModVolAdj.pivot_table(index=['SEGID','FAC_WDAVG','aadtAdjFactor'], columns='YEAR', values='modAADT')
dfModVolAdj_pvModAadt.reset_index(inplace=True)

dfModVolAdj = pd.read_csv('intermediate/model-volumes-adjusted.csv')
dfModVolAdj['YEAR'] = dfModVolAdj['YEAR'].astype(str)
dfModVolAdj['YEAR'] = 'M' + dfModVolAdj['YEAR'] + 'ADJ'

dfModVolAdj_pvModAadtAdj = dfModVolAdj.pivot_table(index=['SEGID','FAC_WDAVG','aadtAdjFactor'], columns='YEAR', values='modAADTadj')
dfModVolAdj_pvModAadtAdj.reset_index(inplace=True)

dfModVolAdj = pd.DataFrame.merge(dfModVolAdj_pvModAadt, dfModVolAdj_pvModAadtAdj, on=('SEGID','FAC_WDAVG','aadtAdjFactor'))

dfModVolAdj['ADJ2023'] = 0
dfModVolAdj['ADJ2028'] = 0
dfModVolAdj['ADJ2032'] = 0
dfModVolAdj['ADJ2042'] = 0
dfModVolAdj['ADJ2050'] = 0

dfModVolAdj

YEAR,SEGID,FAC_WDAVG,aadtAdjFactor,M2019,M2023,M2028,M2032,M2042,M2050,M2019ADJ,M2023ADJ,M2028ADJ,M2032ADJ,M2042ADJ,M2050ADJ,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050
0,0006_146.9,0.9840,-18.0,1535.0,1671.0,1738.0,1892.0,2120.0,2409.0,1517.0,1653.0,1720.0,1874.0,2102.0,2391.0,0,0,0,0,0
1,0006_149.9,0.9840,856.0,1585.0,1799.0,2012.0,2075.0,2652.0,5357.0,2441.0,2655.0,2868.0,2931.0,3508.0,6213.0,0,0,0,0,0
2,0006_150.6,0.9840,657.0,1784.0,1989.0,2209.0,2281.0,2891.0,5598.0,2441.0,2646.0,2866.0,2938.0,3548.0,6255.0,0,0,0,0,0
3,0006_152.6,0.9840,-119.0,2536.0,2719.0,2988.0,3080.0,3756.0,6225.0,2417.0,2600.0,2869.0,2961.0,3637.0,6106.0,0,0,0,0,0
4,0006_152.9,0.9593,-124.0,3883.0,4230.0,4700.0,4991.0,5675.0,8072.0,3759.0,4106.0,4576.0,4867.0,5551.0,7948.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3717,WFRC_8259,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
3718,WFRC_8260,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
3719,WFRC_8261,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
3720,WFRC_8263,1.0924,2427.0,4245.0,5503.0,6511.0,6667.0,6888.0,7125.0,6672.0,7930.0,8938.0,9094.0,9315.0,9552.0,0,0,0,0,0


In [9]:
# join segment data and forecast data
sdfSegmentsWData = pd.DataFrame.merge(dfModVolAdj, sdfSegments, on='SEGID', how='right')
sdfSegmentsWData.fillna(0,inplace=True)

# export
sdfSegmentsWData.spatial.to_featureclass('_site/data/ForecastSegments.shp')

'e:\\GitHub\\Traffic-Volume-Forecasts\\_site\\data\\ForecastSegments.shp'

In [8]:
# convert csvs to jsons
lstSegidFiles = ['aadt', 'linear-forecasts', 'model-volumes-adjusted', 'previous-forecasts']
lstNonSegidFiles = ['aadt-sources', 'previous-forecasts-sources', 'projection-groups']

for file in lstSegidFiles + lstNonSegidFiles:
    # Read CSV file
    pd.read_csv('intermediate/' + file + '.csv').to_json('_site/data/' + file + '.json', orient='records')